In [1]:
import os
import pandas as pd


In [2]:
'''Get CSV to pandas df'''

def get_csv_file_to_df():
    '''This function gets a csv file and makes a pandas df out of it
    OUTPUT: df of the csv file'''

    directory = r'C:\Planetary_Solver\input'

    if os.path.exists(directory):
        files = os.listdir(directory)
        if len(files) > 0:
            print(f'there are {len(files)} files: {files}')
            csv_file_path = os.path.join(directory, files[0])
            df = pd.read_csv(csv_file_path)

    return df

df = get_csv_file_to_df()


there are 1 files: ['data.csv']


In [7]:
# df.columns
cols_to_drop = ['intentType', 'proposal_type', 'jpegURL', 'dataURL', 'obs_title', 'srcDen', 'target_classification', 'dataRights', 'mtFlag']
df_dropped   = df.drop(columns = cols_to_drop)

df_dropped

# ['intentType', 'obs_collection', 'provenance_name', 'instrument_name',
# 'project', 'filters', 'wavelength_region', 'target_name',
# 'target_classification', 'obs_id', 's_ra', 's_dec', 'dataproduct_type',
# 'proposal_pi', 'calib_level', 't_min', 't_max', 't_exptime', 'em_min',
# 'em_max', 'obs_title', 't_obs_release', 'proposal_id', 'proposal_type',
# 'sequence_number', 's_region', 'jpegURL', 'dataURL', 'dataRights',
# 'mtFlag', 'srcDen', 'obsid', 'distance']



,obs_collection,provenance_name,instrument_name,project,filters,wavelength_region,target_name,obs_id,s_ra,s_dec,...,t_max,t_exptime,em_min,em_max,t_obs_release,proposal_id,sequence_number,s_region,obsid,distance
0,TESS,SPOC,Photometer,TESS,TESS,Optical,TESS FFI,tess-s0015-4-1,213.663757,59.235773,...,58736.892960,1425.599379,600.0,1000.0,58756.333333,NaN,15,POLYGON 227.56190400 55.89237000 210.04086200 ...,27507612,0.000000
1,TESS,SPOC,Photometer,TESS,TESS,Optical,TESS FFI,tess-s0016-4-2,212.304021,60.074941,...,58762.809899,1425.599358,600.0,1000.0,58782.333333,NaN,16,POLYGON 229.37435900 60.01041400 214.07446900 ...,27545565,0.000000
2,TESS,SPOC,Photometer,TESS,TESS,Optical,TESS FFI,tess-s0022-3-2,211.552857,53.479178,...,58925.981657,1425.599393,600.0,1000.0,58945.000000,NaN,22,POLYGON 197.21155800 53.68805800 213.62208000 ...,27347175,0.000000
3,TESS,SPOC,Photometer,TESS,TESS,Optical,TESS FFI,tess-s0023-3-1,214.087627,52.092974,...,58954.377214,1425.599383,600.0,1000.0,58973.000000,NaN,23,POLYGON 201.94384300 49.35823100 209.00702800 ...,27386995,0.000000
4,TESS,SPOC,Photometer,TESS,TESS,Optical,TESS FFI,tess-s0049-3-2,216.262733,51.016483,...,59663.814680,475.199787,600.0,1000.0,59695.000000,NaN,49,POLYGON 202.748482 50.900067 217.401787 59.438...,83168230,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,JWST,APT,MIRI/IMAGE,JWST,F2100W,Infrared,M101-H27+H67,jw03429007001_xx10d_00001_miri,210.579739,54.340500,...,NaN,152.627000,18500.0,23500.0,NaN,3429,13,POLYGON 210.55183852133317 54.35240034966904 2...,150099531,419.716239
1996,JWST,APT,MIRI/IMAGE,JWST,F770W,Infrared,M101-H27+H67,jw03429007001_xx102_00002_miri,210.579739,54.340500,...,NaN,208.128000,6600.0,8800.0,NaN,3429,2,POLYGON 210.5518522863841 54.35072771721525 21...,150099511,420.644776
1997,JWST,APT,MIRI/IMAGE,JWST,F1000W,Infrared,M101-H27+H67,jw03429007001_xx106_00002_miri,210.579739,54.340500,...,NaN,291.379000,9000.0,11000.0,NaN,3429,6,POLYGON 210.5518522863841 54.35072771721525 21...,150099518,420.644776
1998,JWST,APT,MIRI/IMAGE,JWST,F1130W,Infrared,M101-H27+H67,jw03429007001_xx10a_00002_miri,210.579739,54.340500,...,NaN,652.134000,10950.0,11650.0,NaN,3429,10,POLYGON 210.5518522863841 54.35072771721525 21...,150099525,420.644776
